Se ha encontrado una nueva fuente, GreeNC, que posee lncRNAs de plantas (http://greenc.sciencedesigners.com).
Se va a realizar el conteo para compararlo contra CantataDB.

In [1]:
import os
import mysql.connector
import pandas as pd

#método para leer los primeros num transcriptomas de un archivo fasta en greenc
def num_transcriptomas_greenc(archivo):
    num = 0
    f = open(archivo, "r")
    for linea in f:
        if linea.startswith(">"):
            num += 1
    return num

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="root",
   passwd=""
)

cur = conn.cursor()
cur.execute("DELETE FROM conteo_lncrna_especie WHERE fuente = 'GreeNC'")
conn.commit()

directory = '.\\httpgreenc.sciencedesigners.com'

for filename in os.listdir(directory):
    if filename.endswith(".fasta"):
        especie = os.path.splitext(filename)[0]
        query = "INSERT INTO conteo_lncrna_especie (fuente, especie, conteo_lncrna) VALUES (%s, %s, %s)"
        conteo = num_transcriptomas_greenc(directory + "\\" + filename)
        cur.execute(query, ('GreeNC', especie, conteo))
conn.commit()

df = pd.read_sql_query("select * from conteo_lncrna_especie WHERE fuente = 'GreeNC' order by 4 desc", conn)
display(df)

conn.close()

,fuente,especie,conteo_ncRNA,conteo_lncRNA,conteo_lincRNA,conteo_pct
0,GreeNC,Triticum_aestivum,None,38820,None,None
1,GreeNC,Zea_mays,None,18110,None,None
2,GreeNC,Physcomitrella_patens,None,9690,None,None
3,GreeNC,Medicago_truncatula,None,9676,None,None
4,GreeNC,Glycine_max,None,6689,None,None
5,GreeNC,Solanum_tuberosum,None,6680,None,None
6,GreeNC,Amborella_trichopoda,None,5698,None,None
7,GreeNC,Brachypodium_distachyon,None,5584,None,None
8,GreeNC,Populus_trichocarpa,None,5569,None,None
9,GreeNC,Sorghum_bicolor,None,5305,None,None


Ahora comparamos los resultados contra ensembl y cantatadb

In [2]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="root",
   passwd=""
)
print("CONTEO CANTATADB")
df = pd.read_sql_query("SELECT c1.especie, c1.conteo_pct, c2.conteo_lncRNA FROM conteo_lncrna_especie c1, conteo_lncrna_especie c2 WHERE c1.especie = REPLACE(c2.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c2.fuente = 'CantataDB' AND c1.especie != 'Zea mays' ORDER BY 3 DESC, 2 DESC LIMIT 10", conn)
display(df)
print("CONTEO GREENC")
df = pd.read_sql_query("SELECT c1.especie, c1.conteo_pct, c2.conteo_lncRNA FROM conteo_lncrna_especie c1, conteo_lncrna_especie c2 WHERE c1.especie = REPLACE(c2.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c2.fuente = 'GreeNC' AND c1.especie != 'Zea mays' ORDER BY 3 DESC, 2 DESC LIMIT 10", conn)
display(df)
print("CONTEO CONJUNTO")
df = pd.read_sql_query("SELECT c1.especie, c1.conteo_pct, c2.conteo_lncRNA as conteo_cantatadb, c3.conteo_lncRNA as conteo_greenc, CASE WHEN IFNULL(c2.conteo_lncRNA, 0) > IFNULL(c3.conteo_lncRNA, 0) THEN c2.conteo_lncRNA ELSE c3.conteo_lncRNA END as conteo_lncRNA, CASE WHEN IFNULL(c2.conteo_lncRNA, 0) > IFNULL(c3.conteo_lncRNA, 0) THEN c2.fuente ELSE c3.fuente END as fuente FROM conteo_lncrna_especie c1 LEFT JOIN conteo_lncrna_especie c2 ON c1.especie = REPLACE(c2.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c2.fuente = 'CantataDB' LEFT JOIN conteo_lncrna_especie c3 ON c1.especie = REPLACE(c3.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c3.fuente = 'GreeNC' WHERE c1.fuente = 'Ensembl' AND c1.especie != 'Zea mays' ORDER BY 5 DESC, 2 DESC LIMIT 20", conn)
display(df)

conn.close()

CONTEO CANTATADB


,especie,conteo_pct,conteo_lncRNA
0,Brassica napus,101040,12010
1,Oryza rufipogon,37071,10261
2,Trifolium pratense,39917,10179
3,Manihot esculenta,33044,9504
4,Oryza nivara,36313,8955
5,Brassica rapa,41018,8501
6,Hordeum vulgare,37705,7970
7,Arabidopsis lyrata,32667,7593
8,Cucumis sativus,23780,7348
9,Brassica oleracea,59220,7338


CONTEO GREENC


,especie,conteo_pct,conteo_lncRNA
0,Triticum aestivum,107545,38820
1,Physcomitrella patens,32447,9690
2,Medicago truncatula,50444,9676
3,Glycine max,55897,6689
4,Solanum tuberosum,39021,6680
5,Amborella trichopoda,27313,5698
6,Brachypodium distachyon,34310,5584
7,Populus trichocarpa,41335,5569
8,Sorghum bicolor,34118,5305
9,Oryza sativa Japonica Group,35821,5237


CONTEO CONJUNTO


,especie,conteo_pct,conteo_cantatadb,conteo_greenc,conteo_lncRNA,fuente
0,Triticum aestivum,107545,NaN,38820.0,38820,GreeNC
1,Brassica napus,101040,12010.0,NaN,12010,CantataDB
2,Oryza rufipogon,37071,10261.0,NaN,10261,CantataDB
3,Trifolium pratense,39917,10179.0,NaN,10179,CantataDB
4,Physcomitrella patens,32447,1498.0,9690.0,9690,GreeNC
5,Medicago truncatula,50444,3590.0,9676.0,9676,GreeNC
6,Manihot esculenta,33044,9504.0,3468.0,9504,CantataDB
7,Oryza nivara,36313,8955.0,NaN,8955,CantataDB
8,Brassica rapa,41018,8501.0,NaN,8501,CantataDB
9,Hordeum vulgare,37705,7970.0,NaN,7970,CantataDB


La data debe ser balanceada, por lo tanto la cantidad de casos positivos (lncRNA) y negativos (PCT) debe ser igual, y también debe ser igual la cantidad de data tomada de cada especie. Ya que siempre podemos asumir que existirá una mayor cantidad de PCTs, el límite vendrá dado por la especie que posea la menor cantidad de lncRNA.

In [5]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="root",
   passwd=""
)
print("SOLO CANTATADB")
df = pd.read_sql_query("SELECT t.especie, t.conteo_pct, t.conteo_lncRNA, t.rank as num_especies, t.conteo_lncRNA * t.rank as conteo_casos_positivos FROM (SELECT t.*, @rownum := @rownum + 1 as rank FROM (SELECT c1.especie, c1.conteo_pct, c2.conteo_lncRNA FROM conteo_lncrna_especie c1, conteo_lncrna_especie c2 WHERE c1.especie = REPLACE(c2.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c2.fuente = 'CantataDB' AND c1.especie != 'Zea mays' ORDER BY 3 DESC) t, (SELECT @rownum := 0) r) t ORDER BY 5 DESC LIMIT 10", conn)
display(df)
print("SOLO GREENC")
df = pd.read_sql_query("SELECT t.especie, t.conteo_pct, t.conteo_lncRNA, t.rank as num_especies, t.conteo_lncRNA * t.rank as conteo_casos_positivos FROM (SELECT t.*, @rownum := @rownum + 1 as rank FROM (SELECT c1.especie, c1.conteo_pct, c2.conteo_lncRNA FROM conteo_lncrna_especie c1, conteo_lncrna_especie c2 WHERE c1.especie = REPLACE(c2.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c2.fuente = 'GreeNC' AND c1.especie != 'Zea mays' ORDER BY 3 DESC) t, (SELECT @rownum := 0) r) t ORDER BY 5 DESC LIMIT 10", conn)
display(df)
print("CANTATADB + GREENC")
df = pd.read_sql_query("SELECT t.especie, t.conteo_pct, t.conteo_lncRNA, t.rank as num_especies, t.conteo_lncRNA * t.rank as conteo_casos_positivos, t.fuente FROM (SELECT t.*, @rownum := @rownum + 1 as rank FROM (SELECT c1.especie, c1.conteo_pct, c2.conteo_lncRNA as conteo_cantatadb, c3.conteo_lncRNA as conteo_greenc, CASE WHEN IFNULL(c2.conteo_lncRNA, 0) > IFNULL(c3.conteo_lncRNA, 0) THEN c2.conteo_lncRNA ELSE c3.conteo_lncRNA END as conteo_lncRNA, CASE WHEN IFNULL(c2.conteo_lncRNA, 0) > IFNULL(c3.conteo_lncRNA, 0) THEN c2.fuente ELSE c3.fuente END as fuente FROM conteo_lncrna_especie c1 LEFT JOIN conteo_lncrna_especie c2 ON c1.especie = REPLACE(c2.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c2.fuente = 'CantataDB' LEFT JOIN conteo_lncrna_especie c3 ON c1.especie = REPLACE(c3.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c3.fuente = 'GreeNC' WHERE c1.fuente = 'Ensembl' AND c1.especie != 'Zea mays' ORDER BY 5 DESC) t, (SELECT @rownum := 0) r) t ORDER BY 5 DESC LIMIT 10", conn)
display(df)

conn.close()

SOLO CANTATADB


,especie,conteo_pct,conteo_lncRNA,num_especies,conteo_casos_positivos
0,Setaria italica,35831,4208,23.0,96784.0
1,Populus trichocarpa,41335,4322,22.0,95084.0
2,Arabidopsis thaliana,27628,4373,21.0,91833.0
3,Vitis vinifera,29927,4542,20.0,90840.0
4,Solanum lycopersicum,33697,4716,19.0,89604.0
5,Theobroma cacao,29188,5256,17.0,89352.0
6,Chlamydomonas reinhardtii,17743,3425,26.0,89050.0
7,Brachypodium distachyon,34310,4945,18.0,89010.0
8,Amborella trichopoda,27313,5511,16.0,88176.0
9,Solanum tuberosum,39021,5790,15.0,86850.0


SOLO GREENC


,especie,conteo_pct,conteo_lncRNA,num_especies,conteo_casos_positivos
0,Prunus persica,26873,3301,18.0,59418.0
1,Solanum lycopersicum,33697,3440,17.0,58480.0
2,Arabidopsis thaliana,27628,3008,19.0,57152.0
3,Musa acuminata,36519,4071,14.0,56994.0
4,Manihot esculenta,33044,3468,16.0,55488.0
5,Gossypium raimondii,38208,4216,13.0,54808.0
6,Setaria italica,35831,3492,15.0,52380.0
7,Oryza sativa Japonica Group,35821,5237,10.0,52370.0
8,Theobroma cacao,29188,4268,12.0,51216.0
9,Vitis vinifera,29927,2526,20.0,50520.0


CANTATADB + GREENC


,especie,conteo_pct,conteo_lncRNA,num_especies,conteo_casos_positivos,fuente
0,Oryza sativa Japonica Group,35821,5237,25.0,130925.0,GreeNC
1,Setaria italica,35831,4208,30.0,126240.0,CantataDB
2,Musa acuminata,36519,4071,31.0,126201.0,GreeNC
3,Theobroma cacao,29188,5256,24.0,126144.0,CantataDB
4,Vitis vinifera,29927,4542,27.0,122634.0,CantataDB
5,Solanum lycopersicum,33697,4716,26.0,122616.0,CantataDB
6,Populus trichocarpa,41335,5569,22.0,122518.0,GreeNC
7,Arabidopsis thaliana,27628,4373,28.0,122444.0,CantataDB
8,Gossypium raimondii,38208,4216,29.0,122264.0,GreeNC
9,Sorghum bicolor,34118,5305,23.0,122015.0,GreeNC


Analizando la información:
- Si se utiliza como fuente de datos solamente CantataDB, la mayor cantidad de datos viene dada al tomar 23 especies, con lo que obtendríamos más de 96 mil transcriptomas de casos positivospara todas las especies.
- Si se utiliza como fuente de datos solamente GreeNC, al tomar 18 especies obtenemos casi 60 mil casos positivos. Es un número menor de casos positivos y de especies que en el caso de CantataDB.
- Si se combinan ambas fuentes (con la restricción de que la data positiva - lncRNA - de una especie puede venir solamente de una de las dos fuentes),  al tomar 25 especies tenemos una muestra de 130 mil casos positivos. Más que al tomarlas por separado.
- Lo interesante es que al tomar 30 especies, tenemos 126 mil casos positivos, solamente 4 mil menos que el menor caso, pero con la ventaja de un mayor número de especies. Esto es favorable para la generalización del modelo. Se opta por esta opción.

Entonces finalmente se presenta la lista de especies que se utilizará, y su origen. Se tomarán 30 especies, teniendo cada una de ellas 4208 transcriptomas para casos positivos (lncRNA) y 4208 transcriptomas para casos negativos (PCT), lo cual en total forma un dataset de 252480 transcriptomas.

In [4]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="root",
   passwd=""
)
df = pd.read_sql_query("SELECT c1.especie, c1.conteo_pct, CASE WHEN IFNULL(c2.conteo_lncRNA, 0) > IFNULL(c3.conteo_lncRNA, 0) THEN c2.conteo_lncRNA ELSE c3.conteo_lncRNA END as conteo_lncRNA, CASE WHEN IFNULL(c2.conteo_lncRNA, 0) > IFNULL(c3.conteo_lncRNA, 0) THEN c2.fuente ELSE c3.fuente END as fuente FROM conteo_lncrna_especie c1 LEFT JOIN conteo_lncrna_especie c2 ON c1.especie = REPLACE(c2.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c2.fuente = 'CantataDB' LEFT JOIN conteo_lncrna_especie c3 ON c1.especie = REPLACE(c3.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c3.fuente = 'GreeNC' WHERE c1.fuente = 'Ensembl' AND c1.especie != 'Zea mays' ORDER BY 3 DESC, 2 DESC LIMIT 30", conn)
display(df)

conn.close()

,especie,conteo_pct,conteo_lncRNA,fuente
0,Triticum aestivum,107545,38820,GreeNC
1,Brassica napus,101040,12010,CantataDB
2,Oryza rufipogon,37071,10261,CantataDB
3,Trifolium pratense,39917,10179,CantataDB
4,Physcomitrella patens,32447,9690,GreeNC
5,Medicago truncatula,50444,9676,GreeNC
6,Manihot esculenta,33044,9504,CantataDB
7,Oryza nivara,36313,8955,CantataDB
8,Brassica rapa,41018,8501,CantataDB
9,Hordeum vulgare,37705,7970,CantataDB
